In [2]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# Загрузка данных

In [3]:
%%capture
!wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx

In [4]:
data = pd.read_excel('data_ford_price.xlsx') 

FileNotFoundError: [Errno 2] No such file or directory: 'data_ford_price.xlsx'

In [179]:
import numpy as np
data[data.isnull().any(1)]

C:\Users\User\AppData\Local\Temp\ipykernel_1124\253931226.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  data[data.isnull().any(1)]


,price,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather
4,13865,2010,3,8,166062,clean,automatic,4wd,NaN,49.210949,-123.114720,NaN
8,18750,2011,2,6,77000,clean,automatic,rwd,NaN,33.698900,-84.745400,59.0
12,1700,2001,1,8,275000,clean,automatic,4wd,NaN,44.549400,-90.611200,45.0
18,32500,2015,3,6,73300,clean,automatic,4wd,full-size,49.184885,-122.801111,NaN
21,32700,2015,3,6,88500,clean,automatic,4wd,full-size,50.039078,-125.271885,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
6991,6600,2010,2,8,101215,clean,automatic,rwd,NaN,43.098946,-76.097853,53.0
7007,28500,2014,3,8,40000,clean,automatic,4wd,NaN,36.562900,-82.653200,59.0
7008,14995,2010,3,8,0,clean,automatic,NaN,full-size,32.788557,-117.098278,59.0
7011,30750,2016,4,6,66000,clean,automatic,4wd,NaN,37.132840,-95.785580,51.0


In [180]:
columns_to_change = ['cylinders', 'title_status', 'transmission', 'drive', 'size']

In [181]:
y = data['price']
X = data.drop(columns='price')


In [182]:
data_test = pd.get_dummies(data, columns=columns_to_change, dummy_na=True)
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7017 entries, 0 to 7016
Data columns (total 33 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   price                   7017 non-null   int64  
 1   year                    7017 non-null   int64  
 2   condition               7017 non-null   int64  
 3   odometer                7017 non-null   int64  
 4   lat                     7017 non-null   float64
 5   long                    7017 non-null   float64
 6   weather                 6837 non-null   float64
 7   cylinders_3.0           7017 non-null   uint8  
 8   cylinders_4.0           7017 non-null   uint8  
 9   cylinders_5.0           7017 non-null   uint8  
 10  cylinders_6.0           7017 non-null   uint8  
 11  cylinders_8.0           7017 non-null   uint8  
 12  cylinders_10.0          7017 non-null   uint8  
 13  cylinders_nan           7017 non-null   uint8  
 14  title_status_clean      7017 non-null   

# Предобработка

In [161]:
#lr = LinearRegression()
#lr.fit(X,y)

_______________________
Сторонний пример

In [162]:
from sklearn.preprocessing  import LabelBinarizer
 
lb = LabelBinarizer()
 
education = ['нет', 'начальное', 'среднее', 'BSc', 'MSc', 'начальное', 'PhD']
lb.fit(education)
 
print('категории:', lb.classes_) 
lb.transform(['нет', 'MSc'])

категории: ['BSc' 'MSc' 'PhD' 'начальное' 'нет' 'среднее']


array([[0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0]])

___________________________________

In [163]:
columns_to_change = ['cylinders', 'title_status', 'transmission', 'drive', 'size']
 
for column in columns_to_change:
 print('Число уникальных значений призкака {}: '.format(column), data[column].nunique())

Число уникальных значений призкака cylinders:  6
Число уникальных значений призкака title_status:  5
Число уникальных значений призкака transmission:  3
Число уникальных значений призкака drive:  3
Число уникальных значений призкака size:  4


In [164]:
from sklearn.preprocessing import OneHotEncoder
 
one_hot_encoder = OneHotEncoder()
 
# 'учим' и сразу применяем преобразование к выборке, результат переводим в массив
data_onehot = one_hot_encoder.fit_transform(data[columns_to_change]).toarray() 
# запишем полученные названия новых колонок в отдельную переменную
column_names = one_hot_encoder.get_feature_names_out((columns_to_change))
print(column_names)

['cylinders_3' 'cylinders_4' 'cylinders_5' 'cylinders_6' 'cylinders_8'
 'cylinders_10' 'title_status_clean' 'title_status_lien'
 'title_status_missing' 'title_status_rebuilt' 'title_status_salvage'
 'transmission_automatic' 'transmission_manual' 'transmission_other'
 'drive_4wd' 'drive_fwd' 'drive_rwd' 'drive_nan' 'size_compact'
 'size_full-size' 'size_mid-size' 'size_sub-compact' 'size_nan']


In [165]:
data_onehot = pd.DataFrame(data_onehot, columns=column_names, index=data.index)

In [166]:
data = pd.concat([data, data_onehot], axis=1)
data.drop(columns=columns_to_change, axis=1, inplace=True)
data

,price,year,condition,odometer,lat,long,weather,cylinders_3,cylinders_4,cylinders_5,...,transmission_other,drive_4wd,drive_fwd,drive_rwd,drive_nan,size_compact,size_full-size,size_mid-size,size_sub-compact,size_nan
0,43900,2016,4,43500,36.471500,-82.483400,59.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,15490,2009,2,98131,40.468826,-74.281734,52.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2495,2002,2,201803,42.477134,-82.949564,45.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1300,2000,1,170305,40.764373,-82.349503,49.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,13865,2010,3,166062,49.210949,-123.114720,NaN,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7012,22500,2015,3,23500,32.680700,-117.169800,59.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
7013,5975,2005,2,0,38.213303,-85.785762,50.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
7014,9999,2006,3,161514,37.609783,-120.995406,59.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
7015,10900,2011,2,164000,43.140600,-93.385000,47.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [167]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7017 entries, 0 to 7016
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   price                   7017 non-null   int64  
 1   year                    7017 non-null   int64  
 2   condition               7017 non-null   int64  
 3   odometer                7017 non-null   int64  
 4   lat                     7017 non-null   float64
 5   long                    7017 non-null   float64
 6   weather                 6837 non-null   float64
 7   cylinders_3             7017 non-null   float64
 8   cylinders_4             7017 non-null   float64
 9   cylinders_5             7017 non-null   float64
 10  cylinders_6             7017 non-null   float64
 11  cylinders_8             7017 non-null   float64
 12  cylinders_10            7017 non-null   float64
 13  title_status_clean      7017 non-null   float64
 14  title_status_lien       7017 non-null   

In [107]:
data.iloc[data.isna().index]

,price,year,condition,odometer,lat,long,weather,cylinders_3,cylinders_4,cylinders_5,...,transmission_other,drive_4wd,drive_fwd,drive_rwd,drive_nan,size_compact,size_full-size,size_mid-size,size_sub-compact,size_nan
0,43900,2016,4,43500,36.471500,-82.483400,59.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,15490,2009,2,98131,40.468826,-74.281734,52.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2495,2002,2,201803,42.477134,-82.949564,45.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1300,2000,1,170305,40.764373,-82.349503,49.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,13865,2010,3,166062,49.210949,-123.114720,NaN,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7012,22500,2015,3,23500,32.680700,-117.169800,59.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
7013,5975,2005,2,0,38.213303,-85.785762,50.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
7014,9999,2006,3,161514,37.609783,-120.995406,59.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
7015,10900,2011,2,164000,43.140600,-93.385000,47.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [73]:
data.shape

(7017, 30)